In [80]:
#import required libraries


# Pandas 
import pandas as pd
from pandas import Series,DataFrame 

# Numpy and Matplotlib
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
#sns.set_style('whitegrid')
%matplotlib inline

# Machine Learning 
from sklearn import preprocessing

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
# The error metric. In this case, we will use c-stat (language used in statistics :)) (aka ROC/AUC)
from sklearn.metrics import roc_auc_score


#import lightgbm and xgboost 
import lightgbm as lgb 
import xgboost as xgb 

%pylab inline

#Import the data
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
#y_train = X_train.pop("is_promoted")

Populating the interactive namespace from numpy and matplotlib


In [81]:
train['source']='train'
test['source']='test'
data = pd.concat([train, test],ignore_index=True)
print(train.shape, test.shape, data.shape)

(54808, 15) (23490, 14) (78298, 15)


In [82]:
data.apply(lambda x: sum(x.isnull()))

KPIs_met >80%               0
age                         0
avg_training_score          0
awards_won?                 0
department                  0
education                3443
employee_id                 0
gender                      0
is_promoted             23490
length_of_service           0
no_of_trainings             0
previous_year_rating     5936
recruitment_channel         0
region                      0
source                      0
dtype: int64

In [83]:
data.describe()

,KPIs_met >80%,age,avg_training_score,awards_won?,employee_id,is_promoted,length_of_service,no_of_trainings,previous_year_rating
count,78298.000000,78298.000000,78298.000000,78298.000000,78298.000000,54808.000000,78298.000000,78298.000000,72362.000000
mean,0.354032,34.797619,63.349664,0.023053,39149.500000,0.085170,5.848974,1.253378,3.332219
std,0.478222,7.665928,13.383664,0.150073,22602.830026,0.279137,4.248070,0.606766,1.260982
min,0.000000,20.000000,39.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,29.000000,51.000000,0.000000,19575.250000,0.000000,3.000000,1.000000,3.000000
50%,0.000000,33.000000,60.000000,0.000000,39149.500000,0.000000,5.000000,1.000000,3.000000
75%,1.000000,39.000000,76.000000,0.000000,58723.750000,0.000000,7.000000,1.000000,4.000000
max,1.000000,60.000000,99.000000,1.000000,78298.000000,1.000000,37.000000,10.000000,5.000000


In [84]:
data.apply(lambda x: len(x.unique()))

KPIs_met >80%               2
age                        41
avg_training_score         61
awards_won?                 2
department                  9
education                   4
employee_id             78298
gender                      2
is_promoted                 3
length_of_service          35
no_of_trainings            10
previous_year_rating        6
recruitment_channel         3
region                     34
source                      2
dtype: int64

In [87]:
#Filter categorical variables
categorical_columns = [x for x in data.dtypes.index if data.dtypes[x]=='object']
#Exclude ID cols and source:
categorical_columns = [x for x in categorical_columns if x not in ['employee_id','source']]
#Print frequency of categories
for col in categorical_columns:
    print('\nFrequency of Categories for varible %s'%col)
    print(data[col].value_counts())


Frequency of Categories for varible department
Sales & Marketing    24155
Operations           16112
Procurement          10158
Technology           10149
Analytics             7671
Finance               3627
HR                    3503
Legal                 1484
R&D                   1439
Name: department, dtype: int64

Frequency of Categories for varible education
Bachelor's          52247
Master's & above    21429
Below Secondary      1179
Name: education, dtype: int64

Frequency of Categories for varible gender
m    55092
f    23206
Name: gender, dtype: int64

Frequency of Categories for varible recruitment_channel
other       43524
sourcing    33181
referred     1593
Name: recruitment_channel, dtype: int64

Frequency of Categories for varible region
region_2     17642
region_22     9167
region_7      6825
region_15     3938
region_13     3815
region_26     3271
region_31     2779
region_4      2478
region_27     2369
region_16     2055
region_28     1913
region_11     1886
region_

In [89]:
# Impute previous_year_rating and education with mode
data['previous_year_rating'].fillna(3, inplace=True)
data['education'].fillna("Bachelor's", inplace=True)

In [91]:
data.apply(lambda x: sum(x.isnull()))

KPIs_met >80%               0
age                         0
avg_training_score          0
awards_won?                 0
department                  0
education                   0
employee_id                 0
gender                      0
is_promoted             23490
length_of_service           0
no_of_trainings             0
previous_year_rating        0
recruitment_channel         0
region                      0
source                      0
dtype: int64

In [92]:
# Here is a simple function to show descriptive stats on the categorical variables
def describe_categorical(X):
    """
    Just like .describe(), but returns the results for 
    categorical variables only.
    """
    from IPython.display import display, HTML
    display(HTML(data[data.columns[data.dtypes == "object"]].describe().to_html()))

In [93]:
describe_categorical(data)

,department,education,gender,recruitment_channel,region,source
count,78298,78298,78298,78298,78298,78298
unique,9,3,2,3,34,2
top,Sales & Marketing,Bachelor's,m,other,region_2,train
freq,24155,55690,55092,43524,17642,54808


In [95]:
categorical_variables=["department","region","education","gender","recruitment_channel"]

for variable in categorical_variables:
        # Fill missing data (NaN values) with the word "Missing"
        data[variable].fillna("Missing",inplace=True)
        
        # Create array of dummies
        dummy=pd.get_dummies(data[variable],prefix=variable)
        
        
        # Update X to include dummies and drop the main variable
        data=pd.concat([data,dummy],axis=1)
        data.drop([variable],axis=1,inplace=True)#dropping original above categorical variable

In [96]:
data

,KPIs_met >80%,age,avg_training_score,awards_won?,employee_id,is_promoted,length_of_service,no_of_trainings,previous_year_rating,source,...,region_region_8,region_region_9,education_Bachelor's,education_Below Secondary,education_Master's & above,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing
0,1,35,49,0,65438,0.0,8,1,5.0,train,...,0,0,0,0,1,1,0,0,0,1
1,0,30,60,0,65141,0.0,4,1,5.0,train,...,0,0,1,0,0,0,1,1,0,0
2,0,34,50,0,7513,0.0,7,1,3.0,train,...,0,0,1,0,0,0,1,0,0,1
3,0,39,50,0,2542,0.0,10,2,1.0,train,...,0,0,1,0,0,0,1,1,0,0
4,0,45,73,0,48945,0.0,2,1,3.0,train,...,0,0,1,0,0,0,1,1,0,0
5,0,31,85,0,58896,0.0,7,2,3.0,train,...,0,0,1,0,0,0,1,0,0,1
6,0,31,59,0,20379,0.0,5,1,3.0,train,...,0,0,1,0,0,1,0,1,0,0
7,0,33,63,0,16290,0.0,6,1,3.0,train,...,0,0,0,0,1,0,1,0,0,1
8,0,28,83,0,73202,0.0,5,1,4.0,train,...,0,0,1,0,0,0,1,1,0,0
9,1,32,54,0,28911,0.0,5,1,5.0,train,...,0,0,0,0,1,0,1,0,0,1


In [97]:
# def printall(x,max_rows=10):   #this function to show all coloumns, ensures that there is no compression in the columns
#     from IPython.display import display,HTML
#     display(HTML(x.to_html(max_rows=max_rows)))
    
# printall(X_data)
data.dtypes

KPIs_met >80%                     int64
age                               int64
avg_training_score                int64
awards_won?                       int64
employee_id                       int64
is_promoted                     float64
length_of_service                 int64
no_of_trainings                   int64
previous_year_rating            float64
source                           object
department_Analytics              uint8
department_Finance                uint8
department_HR                     uint8
department_Legal                  uint8
department_Operations             uint8
department_Procurement            uint8
department_R&D                    uint8
department_Sales & Marketing      uint8
department_Technology             uint8
region_region_1                   uint8
region_region_10                  uint8
region_region_11                  uint8
region_region_12                  uint8
region_region_13                  uint8
region_region_14                  uint8


In [99]:
#Drop the columns which have been converted to different types:
#data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

#Drop unnecessary columns:
test.drop(['is_promoted','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

#Export files as modified versions:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [101]:
##Model Building

#Mean based:
mean_sales = train['is_promoted'].mode()

#Define a dataframe with IDs for submission:
base1 = test[['employee_id']]
base1['is_promoted'] = mean_sales

#Export submission file
base1.to_csv("alg0.csv",index=False)

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [102]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#Generic function for making a classification model and accessing performance:
def classification_model(model, data, predictors, outcome):
  #Fit the model:
  model.fit(data[predictors],data[outcome])
  
  #Make predictions on training set:
  predictions = model.predict(data[predictors])
  
  #Print accuracy
  accuracy = metrics.accuracy_score(predictions,data[outcome])
  print ("Accuracy : %s" % "{0:.3%}".format(accuracy))

  #Perform k-fold cross-validation with 5 folds
  kf = KFold(data.shape[0], n_folds=5)
  error = []
  for train, test in kf:
    # Filter training data
    train_predictors = (data[predictors].iloc[train,:])
    
    # The target we're using to train the algorithm.
    train_target = data[outcome].iloc[train]
    
    # Training the algorithm using the predictors and target.
    model.fit(train_predictors, train_target)
    
    #Record error from each cross-validation run
    error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
  print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))

  #Fit the model again so that it can be refered outside the function:
  model.fit(data[predictors],data[outcome])
        


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [132]:
outcome_var = 'is_promoted'
model = LogisticRegression()
predictor_var = ['avg_training_score']
classification_model(model, train,predictor_var,outcome_var)


#Predict on testing data:
test[outcome_var] = model.predict(test[predictor_var])

#Export submission file:
filename = "LogisticRegression.csv"
IDcol = ['employee_id']
IDcol.append(outcome_var)
submission = pd.DataFrame({ z: test[z] for z in IDcol})
submission.to_csv(filename, index=False)

Accuracy : 91.483%
Cross-Validation Score : 91.483%


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [133]:
outcome_var = 'is_promoted'
model = LogisticRegression()
predictor_var = ['avg_training_score','age','length_of_service','previous_year_rating','KPIs_met >80%','gender_f','gender_m','no_of_trainings','awards_won?']
classification_model(model, train,predictor_var,outcome_var)

#Predict on testing data:
test[outcome_var] = model.predict(test[predictor_var])

#Export submission file:
filename = "LogisticRegressionMoreParams.csv"
IDcol = ['employee_id']
IDcol.append(outcome_var)
submission = pd.DataFrame({ z: test[z] for z in IDcol})
submission.to_csv(filename, index=False)

Accuracy : 91.645%
Cross-Validation Score : 91.645%


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [134]:
model = RandomForestClassifier(n_estimators=2000, 
                               oob_score = True, 
                               n_jobs = -1, 
                               random_state = 42,
                               max_features = "auto",
                               min_samples_leaf = 17)
predictor_var = ['avg_training_score','age','length_of_service','previous_year_rating',
                 'KPIs_met >80%','gender_f','gender_m','no_of_trainings','awards_won?']
classification_model(model, train,predictor_var,outcome_var)

#Predict on testing data:
test[outcome_var] = model.predict(test[predictor_var])

#Export submission file:
filename = "RandomForestClassifierAllParams.csv"
IDcol = ['employee_id']
IDcol.append(outcome_var)
submission = pd.DataFrame({ z: test[z] for z in IDcol})
submission.to_csv(filename, index=False)

Accuracy : 92.702%
Cross-Validation Score : 92.543%


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [135]:
#Create a series with feature importances:
featimp = pd.Series(model.feature_importances_, index=predictor_var).sort_values(ascending=False)
print (featimp)

avg_training_score      0.483337
KPIs_met >80%           0.162725
awards_won?             0.101918
previous_year_rating    0.090950
age                     0.077234
length_of_service       0.059862
no_of_trainings         0.008306
gender_f                0.007974
gender_m                0.007695
dtype: float64


In [136]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=2000, learning_rate=0.01, max_depth=3, random_state=0)
predictor_var = ['avg_training_score',
                 'KPIs_met >80%',
                 'awards_won?',
                 'previous_year_rating',
                 'age']
classification_model(model, train,predictor_var,outcome_var)  


#Predict on testing data:
test[outcome_var] = model.predict(test[predictor_var])

#Export submission file:
filename = "GradientBoostingClassifier.csv"
IDcol = ['employee_id']
IDcol.append(outcome_var)
submission = pd.DataFrame({ z: test[z] for z in IDcol})
submission.to_csv(filename, index=False)

Accuracy : 92.596%
Cross-Validation Score : 92.518%


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [137]:
model = xgb.XGBClassifier(seed=42, max_depth=10, learning_rate=0.3, n_estimators=100)
predictor_var = ['avg_training_score',
                 'KPIs_met >80%',
                 'awards_won?',
                 'previous_year_rating',
                 'age']
classification_model(model, train,predictor_var,outcome_var)  

#Predict on testing data:
test[outcome_var] = model.predict(test[predictor_var])

#Export submission file:
filename = "XGBClassifier.csv"
IDcol = ['employee_id']
IDcol.append(outcome_var)
submission = pd.DataFrame({ z: test[z] for z in IDcol})
submission.to_csv(filename, index=False)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy : 93.318%


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

Cross-Validation Score : 92.072%


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [138]:
model = lgb.LGBMClassifier(seed=42, max_depth=10, learning_rate=0.3, n_estimators=100)
predictor_var = ['avg_training_score',
                 'KPIs_met >80%',
                 'awards_won?',
                 'previous_year_rating',
                 'age']
classification_model(model, train,predictor_var,outcome_var)  


#Predict on testing data:
test[outcome_var] = model.predict(test[predictor_var])

#Export submission file:
filename = "LGBMClassifier.csv"
IDcol = ['employee_id']
IDcol.append(outcome_var)
submission = pd.DataFrame({ z: test[z] for z in IDcol})
submission.to_csv(filename, index=False)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy : 92.904%


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, b

Cross-Validation Score : 92.311%


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
